# Question 02


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
device= 'cuda:0'

In [2]:
# Transform to normalize the data and convert it to a tensor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [3]:
# Load the MNIST dataset and apply the transform
mnist_train = torchvision.datasets.MNIST(root='.', train=True, download=True, transform=transform)
mnist_test = torchvision.datasets.MNIST(root='.', train=False, download=True, transform=transform)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [4]:
# Create data loaders for the training and test sets
target_classes = [1, 3, 5, 7, 9]
mnist_train = torch.utils.data.Subset(mnist_train, [i for i, (x, y) in enumerate(mnist_train) if y in target_classes])
mnist_test = torch.utils.data.Subset(mnist_test, [i for i, (x, y) in enumerate(mnist_test) if y in target_classes])
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=32, shuffle=False)

In [5]:
# Define the autoencoder model
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 8, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(8, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 7)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(32, 16, 7),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 8, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(8, 1, 3, stride=2, padding=1, output_padding=1)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [6]:
# Initialize the model, loss function, and optimizer
model = Autoencoder()
model.to(device)

Autoencoder(
  (encoder): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(16, 32, kernel_size=(7, 7), stride=(1, 1))
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(32, 16, kernel_size=(7, 7), stride=(1, 1))
    (1): ReLU()
    (2): ConvTranspose2d(16, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (3): ReLU()
    (4): ConvTranspose2d(8, 1, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
  )
)

In [7]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [8]:
# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Only keep the images with labels in the target classes
        target_classes = [1, 3, 5, 7, 9]
        target_classes = torch.tensor(target_classes).to(device)
        mask = torch.isin(labels, target_classes)
        images = images[mask]
        labels = labels[mask]
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, images)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #if (i+1) % 100 == 0:
    print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

Epoch [1/10], Step [954/954], Loss: 0.0776
Epoch [2/10], Step [954/954], Loss: 0.0418
Epoch [3/10], Step [954/954], Loss: 0.0329
Epoch [4/10], Step [954/954], Loss: 0.0373
Epoch [5/10], Step [954/954], Loss: 0.0373
Epoch [6/10], Step [954/954], Loss: 0.0280
Epoch [7/10], Step [954/954], Loss: 0.0360
Epoch [8/10], Step [954/954], Loss: 0.0322
Epoch [9/10], Step [954/954], Loss: 0.0302
Epoch [10/10], Step [954/954], Loss: 0.0324


## Drop the decoder and stack FCs

In [9]:
class AE(nn.Module):
    def __init__(self):
        super(AE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 8, 3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(8, 16, 3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(16, 32, 3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2)
        )
        self.fc = nn.Linear(3 * 3 * 32, 512)

    def forward(self, x):
            x = self.encoder(x)
            x = x.view(x.size(0), -1)
            x = F.relu(self.fc(x))
            x = F.log_softmax(x, dim=1)
            return x

aut = AE()
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(aut.parameters(), lr=0.001)

for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = aut(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('Epoch %d loss: %.3f' % (epoch + 1, running_loss / len(train_loader)))

print('Finished Training')


Epoch 1 loss: 0.387
Epoch 2 loss: 0.061
Epoch 3 loss: 0.037
Epoch 4 loss: 0.028
Epoch 5 loss: 0.021
Epoch 6 loss: 0.018
Epoch 7 loss: 0.016
Epoch 8 loss: 0.013
Epoch 9 loss: 0.012
Epoch 10 loss: 0.010
Finished Training


In [11]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = aut(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print('Test accuracy: %d %%' % test_accuracy)

Test accuracy: 99 %
